In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bosch-production-line-performance/train_date.csv.zip
/kaggle/input/bosch-production-line-performance/sample_submission.csv.zip
/kaggle/input/bosch-production-line-performance/train_numeric.csv.zip
/kaggle/input/bosch-production-line-performance/test_date.csv.zip
/kaggle/input/bosch-production-line-performance/test_categorical.csv.zip
/kaggle/input/bosch-production-line-performance/test_numeric.csv.zip
/kaggle/input/bosch-production-line-performance/train_categorical.csv.zip


In [2]:
import shutil
shutil.unpack_archive("/kaggle/input/bosch-production-line-performance/train_categorical.csv.zip")
shutil.unpack_archive("/kaggle/input/bosch-production-line-performance/train_numeric.csv.zip")
shutil.unpack_archive("/kaggle/input/bosch-production-line-performance/train_date.csv.zip")

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 16.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=17523363e847abbf7e17935c3a3d4bd7f8f5b771d0f295d752fe355020380f91
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [4]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [19]:
!ls -ltr

total 7530896
---------- 1 root root        263 Dec  1 11:27 __notebook_source__.ipynb
drwxr-xr-x 3 root root       4096 Dec  1 11:46 spark-warehouse
-rw-r--r-- 1 root root 2678343036 Dec  1 12:12 train_categorical.csv
-rw-r--r-- 1 root root 2139901506 Dec  1 12:12 train_numeric.csv
-rw-r--r-- 1 root root 2893367134 Dec  1 12:13 train_date.csv


In [20]:
!uname -a

Linux 32e37ffa6f66 5.15.65+ #1 SMP Thu Nov 10 10:13:28 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux


In [21]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          4
On-line CPU(s) list:             0-3
Thread(s) per core:              2
Core(s) per socket:              2
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:                        0
CPU MHz:                         2199.998
BogoMIPS:                        4399.99
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       64 KiB
L1i cache:                       64 KiB
L2 cache:                        512 KiB
L3 cache:                        55 MiB
NUMA node0 CPU(s):               0-3
Vulnerability 

Bosch Usage
https://www.youtube.com/watch?v=AEx8s-mNcA8

BOSCH IOT
https://youtube.com/playlist?list=PL9Hl-EX9h_nUNb-RUHLeFXimzZ8rvV9TQ

In [5]:
spark = SparkSession.builder.appName('bosch').getOrCreate()
sparkread = spark.read
sparksql = spark.sql
sparkcon = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/01 12:21:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
#loading the data in pyspark
train_categDF = sparkread.csv("/kaggle/working/train_categorical.csv",
                              inferSchema=True,
                              header=True)
train_numDF = sparkread.csv("/kaggle/working/train_numeric.csv",
                              inferSchema=True,
                              header=True)
train_dateDF = sparkread.csv("/kaggle/working/train_date.csv",
                              inferSchema=True,
                              header=True)

In [7]:
sparksql("""CREATE DATABASE IF NOT EXISTS bosch_db""")
sparksql("""USE bosch_db""")
sparksql("""SHOW TABLES""")

DataFrame[namespace: string, tableName: string, isTemporary: boolean]

In [12]:
train_categDF.createOrReplaceTempView("train_category")

In [13]:
train_numDF.createOrReplaceTempView("train_numeric")

In [14]:
train_dateDF.createOrReplaceTempView("train_date")

In [23]:
train_categDF.select('Id',
 'L0_S1_F25',
 'L0_S1_F27',
 'L0_S1_F29',
 'L0_S1_F31',
 'L0_S2_F33',
 'L0_S2_F35',
 'L0_S2_F37',
 'L0_S2_F39',
 'L0_S2_F41',
 'L0_S2_F43',
 'L0_S2_F45',
 'L0_S2_F47',
 'L0_S2_F49',
 'L0_S2_F51',
 'L0_S2_F53',
 'L0_S2_F55',
 'L0_S2_F57',
 'L0_S2_F59',
 'L0_S2_F61',
 'L0_S2_F63',
 'L0_S2_F65',
 'L0_S2_F67',
 'L0_S3_F69',
 'L0_S3_F71',).describe().show()

+-------+------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|summary|                Id|L0_S1_F31|L0_S2_F35|L0_S2_F39|L0_S2_F43|L0_S2_F47|L0_S2_F51|L0_S2_F55|L0_S2_F59|L0_S2_F63|L0_S2_F67|L0_S3_F69|L0_S3_F71|
+-------+------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  count|           1183747|       27|      263|      263|      263|      263|      263|      263|      263|      263|      263|        0|        0|
|   mean|1184049.7421839295|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|
| stddev| 683434.7677251654|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|
|    min|                 4|      T24|     T128|     T128|     T128|     T128|     T128|     T128|     T12

In [8]:
train_categDF.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- L0_S1_F25: timestamp (nullable = true)
 |-- L0_S1_F27: timestamp (nullable = true)
 |-- L0_S1_F29: timestamp (nullable = true)
 |-- L0_S1_F31: string (nullable = true)
 |-- L0_S2_F33: timestamp (nullable = true)
 |-- L0_S2_F35: string (nullable = true)
 |-- L0_S2_F37: timestamp (nullable = true)
 |-- L0_S2_F39: string (nullable = true)
 |-- L0_S2_F41: timestamp (nullable = true)
 |-- L0_S2_F43: string (nullable = true)
 |-- L0_S2_F45: timestamp (nullable = true)
 |-- L0_S2_F47: string (nullable = true)
 |-- L0_S2_F49: timestamp (nullable = true)
 |-- L0_S2_F51: string (nullable = true)
 |-- L0_S2_F53: timestamp (nullable = true)
 |-- L0_S2_F55: string (nullable = true)
 |-- L0_S2_F57: timestamp (nullable = true)
 |-- L0_S2_F59: string (nullable = true)
 |-- L0_S2_F61: timestamp (nullable = true)
 |-- L0_S2_F63: string (nullable = true)
 |-- L0_S2_F65: timestamp (nullable = true)
 |-- L0_S2_F67: string (nullable = true)
 |-- L0_S3_F69: string

In [17]:
sparksql("SHOW TABLES").show()

+---------+--------------+-----------+
|namespace|     tableName|isTemporary|
+---------+--------------+-----------+
|         |train_category|       true|
|         |    train_date|       true|
|         | train_numeric|       true|
+---------+--------------+-----------+



In [19]:
sparksql("""SELECT Id, L0_S11_F314,
       L0_S0_F18, L3_S35_F3896, L0_S12_F350, L3_S36_F3918,
       L0_S0_F20, L3_S30_F3684, L1_S24_F1632, L0_S2_F48,
       L3_S29_F3345, L0_S18_F449, L0_S21_F497, L3_S29_F3433,
       L3_S30_F3764, L0_S1_F24, L3_S30_F3554, L0_S11_F322,
       L3_S30_F3564, L3_S29_F3327, L0_S2_F36, L0_S9_F180,
       L3_S33_F3855, L0_S0_F4, L0_S21_F477, L0_S5_F114,
       L0_S6_F122, L1_S24_F1122, L0_S9_F165, L0_S18_F439,
       L1_S24_F1490, L0_S6_F132, L3_S29_F3379, L3_S29_F3336,
       L0_S3_F80, L3_S30_F3749, L1_S24_F1763, L0_S10_F219,Response
       FROM train_category""").show(2)

AnalysisException: Column 'L0_S11_F314' does not exist. Did you mean one of the following? [train_category.Id, train_category.L0_S11_F311, train_category.L0_S11_F313, train_category.L0_S11_F315, train_category.L0_S11_F317, train_category.L0_S11_F319, train_category.L0_S11_F301, train_category.L0_S11_F303, train_category.L0_S11_F305, train_category.L0_S11_F307, train_category.L0_S11_F309, train_category.L0_S11_F321, train_category.L0_S11_F323, train_category.L0_S11_F325, train_category.L0_S11_F327, train_category.L0_S11_F329, train_category.L0_S15_F414, train_category.L0_S1_F31, train_category.L0_S10_F215, train_category.L0_S10_F217, train_category.L0_S10_F218, train_category.L0_S11_F279, train_category.L0_S11_F281, train_category.L0_S11_F283, train_category.L0_S11_F285, train_category.L0_S11_F287, train_category.L0_S11_F289, train_category.L0_S11_F291, train_category.L0_S11_F293, train_category.L0_S11_F295, train_category.L0_S11_F297, train_category.L0_S11_F299, train_category.L0_S14_F

In [9]:
train_numDF.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- L0_S0_F0: double (nullable = true)
 |-- L0_S0_F2: double (nullable = true)
 |-- L0_S0_F4: double (nullable = true)
 |-- L0_S0_F6: double (nullable = true)
 |-- L0_S0_F8: double (nullable = true)
 |-- L0_S0_F10: double (nullable = true)
 |-- L0_S0_F12: double (nullable = true)
 |-- L0_S0_F14: double (nullable = true)
 |-- L0_S0_F16: double (nullable = true)
 |-- L0_S0_F18: double (nullable = true)
 |-- L0_S0_F20: double (nullable = true)
 |-- L0_S0_F22: double (nullable = true)
 |-- L0_S1_F24: double (nullable = true)
 |-- L0_S1_F28: double (nullable = true)
 |-- L0_S2_F32: double (nullable = true)
 |-- L0_S2_F36: double (nullable = true)
 |-- L0_S2_F40: double (nullable = true)
 |-- L0_S2_F44: double (nullable = true)
 |-- L0_S2_F48: double (nullable = true)
 |-- L0_S2_F52: integer (nullable = true)
 |-- L0_S2_F56: double (nullable = true)
 |-- L0_S2_F60: double (nullable = true)
 |-- L0_S2_F64: double (nullable = true)
 |-- L0_S3_F68: doubl

In [10]:
train_dateDF.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- L0_S0_D1: double (nullable = true)
 |-- L0_S0_D3: double (nullable = true)
 |-- L0_S0_D5: double (nullable = true)
 |-- L0_S0_D7: double (nullable = true)
 |-- L0_S0_D9: double (nullable = true)
 |-- L0_S0_D11: double (nullable = true)
 |-- L0_S0_D13: double (nullable = true)
 |-- L0_S0_D15: double (nullable = true)
 |-- L0_S0_D17: double (nullable = true)
 |-- L0_S0_D19: double (nullable = true)
 |-- L0_S0_D21: double (nullable = true)
 |-- L0_S0_D23: double (nullable = true)
 |-- L0_S1_D26: double (nullable = true)
 |-- L0_S1_D30: double (nullable = true)
 |-- L0_S2_D34: double (nullable = true)
 |-- L0_S2_D38: double (nullable = true)
 |-- L0_S2_D42: double (nullable = true)
 |-- L0_S2_D46: double (nullable = true)
 |-- L0_S2_D50: double (nullable = true)
 |-- L0_S2_D54: double (nullable = true)
 |-- L0_S2_D58: double (nullable = true)
 |-- L0_S2_D62: double (nullable = true)
 |-- L0_S2_D66: double (nullable = true)
 |-- L0_S3_D70: double